## Initialize table for Postgres database

In [1]:
import dataset
import pandas as pd

In [2]:
db_url = "XXXX"

In [3]:
db = dataset.connect(db_url)

In [22]:
db.tables

['mytable', 'police_force', 'twitter_potential_incidents', 'test']

In [10]:
table = db['police_force']

In [11]:
df = pd.DataFrame(table)

In [12]:
df.head()

,id,dates,added_on,links,case_id,city,state,lat,long,title,description,tags,force_rank
0,1,2020-05-30,2021-02-26 20:45:37.445121,['https://www.youtube.com/watch?v=dPTr54DdTdY'...,or-eugene-2,Eugene,Oregon,44.049913,-123.097391,Peaceful protester tear gassed,Footage shows a small cluster of protesters ch...,"['less-lethal', 'protester', 'tear-gas']",Rank 4 - Chemical & Electric
1,2,2020-05-30,2021-02-26 20:45:37.446934,['https://twitter.com/IwriteOK/status/12669077...,or-portland-1,Portland,Oregon,45.515586,-122.676994,Police violently break up peaceful protest,The Police try to break up the peaceful protes...,"['baton', 'beat', 'protester', 'strike']",Rank 1 - Police Presence
2,3,2020-05-30,2021-02-26 20:45:37.447560,['https://www.youtube.com/watch?v=01oWE24O9Zw&...,or-portland-2,Portland,Oregon,45.515468,-122.676571,Officer pepper-sprays protester,An officer shoves and pepper-sprays a proteste...,"['less-lethal', 'pepper-spray', 'protester', '...",Rank 4 - Chemical & Electric
3,4,2020-05-31,2021-02-26 20:45:37.448138,['https://www.youtube.com/watch?v=R7fqWI41vQY'...,or-eugene-3,Eugene,Oregon,44.044728,-123.079238,Reporter shot with tear gas canister,Footage shows police giving curfew dispersal o...,"['journalist', 'less-lethal', 'rubber-bullet',...",Rank 4 - Chemical & Electric
4,5,2020-06-01,2021-02-26 20:45:37.448824,['https://www.reddit.com/r/Eugene/comments/gur...,or-eugene-1,Eugene,Oregon,0.000000,0.000000,Officer shoots projectile from moving vehicle,An officer shoots a projectile out of a moving...,"['less-lethal', 'projectile', 'protester', 'sh...",Rank 5 - Lethal Force


In [16]:
df['force_rank'].value_counts()

Rank 4 - Chemical & Electric    387
Rank 1 - Police Presence        359
Rank 2 - Empty-hand             205
Rank 5 - Lethal Force           160
Rank 3 - Blunt Force            106
Rank 0 - No Police Presence      79
Name: force_rank, dtype: int64

In [57]:
print(list(df['description'][df['force_rank'] == 'Rank 0 - No Police Presence'])[1])

During a protest at which both pro-police and Black Lives Matter protesters were present near West 7th and Lincoln, members of the two protests appear to exchange words in an intersection. Police charge a protester, a 14 year old girl according to the poster, tackle her to the ground, kneel on her and arrest her.


In [59]:
df.to_csv("reddit_data.csv")

In [34]:
import psycopg2

In [52]:
def execute_sql(command, db_url="postgresql://postgres"initentionally removed"flabs31db.cjb51vwbigfl.us-east-1.rds.amazonaws.com"):
    conn = psycopg2.connect(db_url)
    curs = conn.cursor()
    curs.execute(command)
    conn.commit()
    return curs.fetchall()

In [54]:
SEE_TABLE = """SELECT * FROM twitter_potential_incidents"""

print(execute_sql(SEE_TABLE))

InterfaceError: cursor already closed

In [40]:
statement = 'SELECT id_str FROM twitter_potential_incidents ORDER BY id_str DESC LIMIT 1'

conn = psycopg2.connect(db_url)
curs = conn.cursor()
curs.execute(statement)
conn.commit()
maxid = curs.fetchall()[0][0]

In [41]:
maxid

'1365056998241402888'

In [128]:
delete_duplicates = """
DELETE FROM
    twitter_potential_incidents a
        USING twitter_potential_incidents b
WHERE
    a.id < b.id
    AND a.id_str = b.id_str;
"""

execute_sql(delete_duplicates)

ProgrammingError: no results to fetch

In [6]:
df

,id,dates,added_on,links,case_id,city,state,lat,long,title,description,tags,force_rank
0,1,2020-05-30,2021-02-26 20:45:37.445121,['https://www.youtube.com/watch?v=dPTr54DdTdY'...,or-eugene-2,Eugene,Oregon,44.049913,-123.097391,Peaceful protester tear gassed,Footage shows a small cluster of protesters ch...,"['less-lethal', 'protester', 'tear-gas']",Rank 4 - Chemical & Electric
1,2,2020-05-30,2021-02-26 20:45:37.446934,['https://twitter.com/IwriteOK/status/12669077...,or-portland-1,Portland,Oregon,45.515586,-122.676994,Police violently break up peaceful protest,The Police try to break up the peaceful protes...,"['baton', 'beat', 'protester', 'strike']",Rank 1 - Police Presence
2,3,2020-05-30,2021-02-26 20:45:37.447560,['https://www.youtube.com/watch?v=01oWE24O9Zw&...,or-portland-2,Portland,Oregon,45.515468,-122.676571,Officer pepper-sprays protester,An officer shoves and pepper-sprays a proteste...,"['less-lethal', 'pepper-spray', 'protester', '...",Rank 4 - Chemical & Electric
3,4,2020-05-31,2021-02-26 20:45:37.448138,['https://www.youtube.com/watch?v=R7fqWI41vQY'...,or-eugene-3,Eugene,Oregon,44.044728,-123.079238,Reporter shot with tear gas canister,Footage shows police giving curfew dispersal o...,"['journalist', 'less-lethal', 'rubber-bullet',...",Rank 4 - Chemical & Electric
4,5,2020-06-01,2021-02-26 20:45:37.448824,['https://www.reddit.com/r/Eugene/comments/gur...,or-eugene-1,Eugene,Oregon,0.000000,0.000000,Officer shoots projectile from moving vehicle,An officer shoots a projectile out of a moving...,"['less-lethal', 'projectile', 'protester', 'sh...",Rank 5 - Lethal Force
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290,2031,2020-11-26,2021-02-26 15:57:29.528613,['https://twitter.com/chadloder/status/1332242...,ca-losangeles-64,Los Angeles,California,34.080414,-118.164295,Police beat and arrest anti-eviction protesters,Following evictions of squatting families from...,"['arrest', 'baton', 'beat', 'protester', 'stri...",Rank 1 - Police Presence
1291,2032,2020-12-04,2021-02-26 15:57:29.550788,['https://twitter.com/929_julian/status/133753...,mn-minneapolis-39,Minneapolis,Minnesota,44.934225,-93.262402,Police arrest and injure protester,"During a small protest on December 4th, police...","['arrest', 'protester', 'tackle', 'throw']",Rank 1 - Police Presence
1292,2033,2020-12-06,2021-02-26 15:57:29.573944,['https://twitter.com/PplsCityCouncil/status/1...,ca-losangeles-65,Los Angeles,California,34.063557,-118.318640,Protesters outside Mayor's home beaten with ba...,On the 13th day of protests outside the Getty ...,"['baton', 'legal-observer', 'protester', 'stri...",Rank 3 - Blunt Force
1293,2034,2021-01-09,2021-02-26 15:57:29.596777,['https://twitter.com/waterspider__/status/134...,ca-sandiego-9,San Diego,California,32.796060,-117.255302,Police assault and shoot at BLM counter-protes...,At dueling rallies in Pacific Beach between Pr...,"['baton', 'beat', 'less-lethal', 'pepper-ball'...",Rank 5 - Lethal Force


In [8]:
import requests

In [11]:
json = requests.get("https://testingtwitterincidents.herokuapp.com/dashboard/incidents/dstraining")

In [13]:
pd.read_json("https://testingtwitterincidents.herokuapp.com/dashboard/incidents/dstraining")

,id,date,user_name,user_description,user_location,coordinates,geo,incident_id,src,desc,language,force_rank,pending,approved,rejected
0,491,2021-02-28 00:00:00+00:00,Test from the backend,This is a test post from my API client posting...,California,NaN,NaN,1365089193388365056,https://insomnia.rest,This is a test post from my API client posting...,en,Rank 1 - Police Presence,False,False,True
1,452,2021-02-27 00:00:00+00:00,TostevinM,@Reuters Southeast Asia Editor. #JournalismIsN...,Thailand,NaN,NaN,1365547358755299328,https://t.co/ibsuj5op4Y,Myanmar riot police confront a woman in Yangon...,en,Rank 3 - Blunt Force,False,False,True
2,451,2021-02-27 00:00:00+00:00,TostevinM,@Reuters Southeast Asia Editor. #JournalismIsN...,Thailand,NaN,NaN,1365540152295059456,https://t.co/ibsuj5op4Y,Myanmar police are stepping up arrests after m...,en,Rank 1 - Police Presence,False,False,True
3,466,2021-02-27 00:00:00+00:00,washingtonpost,Democracy Dies in Darkness.,"Washington, DC",NaN,NaN,1365488885929828352,http://t.co/Hq7hTYkOPg,FBI focuses on video of Capitol Police officer...,en,Rank 4 - Chemical & Electric,False,True,False
4,354,2021-02-26 00:00:00+00:00,OpIndia_com,"News and opinions website, bringing the right ...",New Delhi,NaN,NaN,1365089193387565056,https://t.co/61RZ89DYwJ,As Mamata Banerjee and Asaduddin Owaisi fight ...,en,Rank 1 - Police Presence,False,True,False
